In [1]:
import tensorflow as tf
from tensorflow.data import Dataset
from dlomix.losses import masked_spectral_distance, masked_pearson_correlation_distance
import wandb
from wandb.keras import WandbCallback

2023-07-27 09:08:10.423345: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-27 09:08:10.540610: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-07-27 09:08:11.362815: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-07-27 09:08:11.362876: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
project_name = "transforming-prosit-big-data"

In [3]:
run = wandb.init(project=project_name)

wandb: Currently logged in as: mamisashvili-lizi (prosit-compms). Use `wandb login --relogin` to force relogin


In [4]:
model_artifact_name = "prosit-compms/transforming-prosit-big-data/model-lucky-dragon-60:v17"

In [5]:
model = run.use_artifact(model_artifact_name)

In [6]:
model_dir = model.download()

wandb: Downloading large artifact model-lucky-dragon-60:v17, 507.30MB. 4 files... 
wandb:   4 of 4 files downloaded.  
Done. 0:0:1.2


In [7]:
model_dir

'./artifacts/model-lucky-dragon-60:v17'

In [8]:
from dlomix.data import IntensityDataset

TRAIN_DATAPATH = 'https://raw.githubusercontent.com/wilhelm-lab/dlomix-resources/main/example_datasets/Intensity/proteomeTools_train_val.csv'
BATCH_SIZE = 64
int_data = IntensityDataset(data_source=TRAIN_DATAPATH, seq_length=30,
                            collision_energy_col='collision_energy', batch_size=BATCH_SIZE, val_ratio=0.2, test=False)

2023-07-21 12:12:26.060406: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-21 12:12:29.173942: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 27880 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:21:00.0, compute capability: 8.6
2023-07-21 12:12:29.174491: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 43648 MB memory:  -> device: 1, name: NVIDIA A40, pci bus id: 0000:81:00.0, compute capability: 8.6
2023-07-21 12:12:29.174852: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 43648 MB

In [15]:
from tensorflow.data import Dataset

train_data = Dataset.load(
    "/cmnfs/home/l.mamisashvili/transforming-prosit/notebooks/train_data"
)
val_data = Dataset.load(
    "/cmnfs/home/l.mamisashvili/transforming-prosit/notebooks/val_data"
)

In [9]:
model = tf.keras.models.load_model(model_dir, custom_objects={
    "masked_spectral_distance": masked_spectral_distance,
    "masked_pearson_correlation_distance": masked_pearson_correlation_distance
})

In [10]:
model.summary()

Model: "prosit_simple_intensity_predictor"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 string_lookup (StringLookup  multiple                 0         
 )                                                               
                                                                 
 positional_embedding (Posit  multiple                 11264     
 ionalEmbedding)                                                 
                                                                 
 meta_encoder (MetaEncoder)  multiple                  4096      
                                                                 
 transformer_encoder (Transf  multiple                 33675328  
 ormerEncoder)                                                   
                                                                 
 flatten (Flatten)           multiple                  0         
                                 

In [13]:
from keras.callbacks import EarlyStopping
cb_wandb = WandbCallback()

callback_earlystopping = EarlyStopping(
    monitor="val_loss",
    patience=30,
    min_delta=0.0001,
    restore_best_weights=True,
    verbose=1,
)
callbacks = [cb_wandb, callback_earlystopping]

In [ ]:
model.fit(
    train_data,
    validation_data=val_data,
    epochs=200,
    callbacks=callbacks,
)

Epoch 1/200


2023-07-27 09:11:00.907453: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:690] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "NVIDIA A40" frequency: 1740 num_cores: 84 environment { key: "architecture" value: "8.6" } environment { key: "cuda" value: "11020" } environment { key: "cudnn" value: "8100" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 6291456 shared_memory_size_per_multiprocessor: 102400 memory_size: 45768966144 bandwidth: 696096000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }
2023-07-27 09:11:01.664219: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1843/4731 [==========>...................] - ETA: 17:18 - loss: 0.1544 - masked_pearson_correlation_distance: 0.1449

wandb: Network error (ReadTimeout), entering retry loop.


4730/4731 [============================>.] - ETA: 0s - loss: 0.1429 - masked_pearson_correlation_distance: 0.1331

...

4731/4731 [==============================] - 1821s 385ms/step - loss: 0.1392 - masked_pearson_correlation_distance: 0.1336 - val_loss: 0.1922 - val_masked_pearson_correlation_distance: 0.1298
Epoch 7/200
4730/4731 [============================>.] - ETA: 0s - loss: 0.1389 - masked_pearson_correlation_distance: 0.1338

wandb: ERROR Can't save model in the h5py format. The model will be saved as as an W&B Artifact in the 'tf' format.


INFO:tensorflow:Assets written to: /cmnfs/home/l.mamisashvili/transforming-prosit/notebooks/wandb/run-20230727_090815-8m74atgg/files/model-best/assets


INFO:tensorflow:Assets written to: /cmnfs/home/l.mamisashvili/transforming-prosit/notebooks/wandb/run-20230727_090815-8m74atgg/files/model-best/assets
wandb: Adding directory to artifact (/cmnfs/home/l.mamisashvili/transforming-prosit/notebooks/wandb/run-20230727_090815-8m74atgg/files/model-best)... Done. 2.5s


4731/4731 [==============================] - 1820s 385ms/step - loss: 0.1389 - masked_pearson_correlation_distance: 0.1338 - val_loss: 0.1905 - val_masked_pearson_correlation_distance: 0.1291
Epoch 8/200
1982/4731 [===========>..................] - ETA: 16:28 - loss: 0.1540 - masked_pearson_correlation_distance: 0.1483

In [12]:
wandb.finish()